In [ ]:
# import pyspark.sql as ps
# import pyspark.sql.functions as F
# import pyspark

import pandas as pd
import numpy as np

from functools import reduce
import copy

import random
from tqdm import tqdm
import datetime

In [ ]:
spark

In [ ]:
df=spark.read.option("header", "true").option("inferSchema", "true").option("sep", ',').option("quote", '"').option("wholeFile", "true").option("multiline", "true").csv('realtime_snapshot.csv')
df.show()

+-------------------+-----------------+------+-------------------+----------------+--------------------+--------------------+-------------+----------------------+-------------------+---------------------+----------+--------+--------+---------------------+-----------------+----+-------------------+--------------------+----------------+-------+------------------------+------------------+-------------------------+------------------+------------------+-------+------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------------+------------+------------+---------+-----------+-----------+
|        UPDATE_DATE|       DEVICE_KEY|MDT_ID|    COLLECTION_DATE|COLLECTION_STATE|                NMEA|                ADAS|ADAS_DISTANCE|ADAS_FORWARD_COLLISION|ADAS_LANE_DEPARTURE|ADAS_PERSON_DETECTION|ADAS_SPEED|ADAS_RPM|ADAS_AEB|ADAS_LAST_UPDATE_DATE|ADAS_INSTALL_DATE| DSM|DSM_DROWSINESS_CODE|DSM_LAST_UP

In [ ]:
df.printSchema()

In [ ]:
df = df.filter(df.COLLECTION_STATE.contains('SUCCESS'))

In [ ]:
# import pyspark.sql.functions as F
# import pyspark

# import pandas as pd
# import numpy as np

# from functools import reduce

# import datetime

# pandas로 파일을 읽어옴
aa=pd.read_excel("SW_테이블.xlsx", sheet_name="테이블", header=None)
ak1 = aa[aa[2].isnull() == False][2]
ak = ak1.to_list()
ok = np.where(ak1 == '컬럼 ID')[0]
ok = ok.tolist()

for m, c in enumerate(['CO2__DEN', 'COLLECT_DATE', 'DEVICE_K', 'HR_REAL', 'KPH', 'VH_ID']):
    for f in np.where(ak1 == c)[0].tolist():
        change = ['CO2_DENSITY', 'COLLECTION_DATE', 'DEVICE_KEY', 'BAND_HB', 'SPEED', 'BUS_VH_ID']    
        ak[f] = change[m]

# 테이블 별 컬럼
table_dict = {}
try:
    for i in range(len(ok)):
        values = ak[ok[i]+1:ok[i+1]-2]
        try:
            values.remove('LINK_ID')
        except:
            continue
        table_dict[ak[ok[i]-2]] = values
except:
    pass

# 고유 value만 추출
table_list = []
for i in table_dict.values():
    for j in i:
        table_list.append(j)
table_set = [i for i in set(table_list)]

now_list = []
for i in ['TRACK_SPEED', 'TRACK_RAPID_ACC', 'TRACK_RAPID_DEC', 'TRACK_NOSTOP', 'TRACK_NEGLECT', 'TRACK_DROWSY', 'MD_SPEED', 'MD_RAPID_ACC', 'MD_RAPID_DEC', 'MD_NOSTOP', 'MD_NEGLECT', 'MD_DROWSY', 'MD_SB_INFO', 'MD_DOOR_INFO', 'MD_CO2_INFO']:
    for k in table_dict[i]:
        now_list.append(k)
now_list = [j for j in set(now_list)]

# 원하는 테이블명 입력
def table_sort(table_name):
    rddFromFile = spark.sparkContext.textFile("")
    df1 = spark.read.option("header","true").format("json").json(rddFromFile)
    df1 = df1.filter(df1.COLLECTION_STATE.contains('SUCCESS'))
    values = table_dict[table_name]
    for v in values:
        if v is 'WORK_DATE':
            df1 = df1.withColumn('WORK_DATE', F.lit(pd.to_datetime(str(datetime.datetime.now())[:19])))
        elif v not in df.columns:
            df1 = df1.withColumn(v,  F.lit(''))
# 없는 컬럼은 추가하고, 'SUCCESS'값만 모은 spark dataframe    
    return df1.select(values)


# table_dict : 딕셔너리 형태의 키-밸류
# table_set : 총 필요한 컬럼들
# now_list : 지금 해야하는 컬럼 칭구들

In [ ]:
df = pd.read_csv('suwon.csv', encoding='UTF-8')

In [ ]:
df.info()

In [ ]:
df['BUS_ROUTE_ID'] = df['BUS_ROUTE_ID'].astype('str')
df['BUS_VH_ID'] = df['BUS_VH_ID'].astype('str')

In [ ]:
for i in range(len(df)):
    if df.loc[i, 'BUS_ROUTE_ID'] != 'nan':
        df.loc[i, 'BUS_ROUTE_ID'] = df.loc[i, 'BUS_ROUTE_ID'][:-2]
    if df.loc[i, 'BUS_VH_ID'] != 'nan':
        df.loc[i, 'BUS_VH_ID'] = df.loc[i, 'BUS_VH_ID'][:-2]

In [ ]:
df30 = []
all_index=pd.date_range(df.COLLECTION_DATE[0], freq='S', periods=47777)
for i in tqdm(range(47777)):
    df1 = df.copy()
    df1.reset_index()
    for j in range(len(df1)):
        new = random.choice(np.arange(start=(df.SPEED[j] - 20.0), stop=(df.SPEED[j] + 20.0), step=0.5))
        df1.loc[j, 'SPEED'] = new if new >= 0.0 else 0.0
    df1['COLLECTION_DATE'] = all_index[i]
    df1['UPDATE_DATE'] = df1['COLLECTION_DATE'] + pd.Timedelta('3second')
    df30.append(df1)
df_org = pd.concat(df30, axis=0)

100%|██████████| 47777/47777 [2:10:13<00:00,  6.11it/s]   


In [ ]:
df_org.to_csv('fake_snapshot_48.csv', header=True, index=False, encoding='UTF-8')

In [ ]:
df_org

In [ ]:
test_df = spark.read.option("header", "true").option("inferSchema", "true").option("sep", ',').option("quote", '"').option("wholeFile", "true").option("multiline", "true").csv('fake_snapshot.csv')

In [ ]:
test_df.show()

+-------------------+-----------------+------+-------------------+----------------+--------------------+--------------------+-------------+----------------------+-------------------+---------------------+----------+--------+--------+---------------------+-----------------+----+-------------------+--------------------+----------------+-------+------------------------+------------------+-------------------------+------------------+------------------+-------+------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+----------------+------------+------------+---------+-----------+-----------+
|        UPDATE_DATE|       DEVICE_KEY|MDT_ID|    COLLECTION_DATE|COLLECTION_STATE|                NMEA|                ADAS|ADAS_DISTANCE|ADAS_FORWARD_COLLISION|ADAS_LANE_DEPARTURE|ADAS_PERSON_DETECTION|ADAS_SPEED|ADAS_RPM|ADAS_AEB|ADAS_LAST_UPDATE_DATE|ADAS_INSTALL_DATE| DSM|DSM_DROWSINESS_CODE

In [ ]:
test_df.coalesce(1).write.mode('append').option('encoding', 'UTF-8').option('charset', "UTF-8").parquet('')

In [ ]:
test_df.write.mode('append').option('encoding', 'UTF-8').option('charset', "UTF-8").json('')

In [ ]:
import json

In [ ]:
test_df.toJSON()

MapPartitionsRDD[66] at toJavaRDD at NativeMethodAccessorImpl.java:0

In [ ]:
fakeRdd = spark.sparkContext.textFile("")
test_df = spark.read.option("header","true").format("json").json(fakeRdd)

In [ ]:
test_df.show()

+--------------------+--------+-------------+----------------------+-------------------+---------------------+---------------------+--------+----------+------+------------------+-------------------------+-------+------------------------+----------------+------------+------------+------------+---------+-----------+-------------------+----------------+-----------------+-----------+----+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+------------------+------------------+------+--------------------+------------------+-------------------+
|                ADAS|ADAS_AEB|ADAS_DISTANCE|ADAS_FORWARD_COLLISION|ADAS_LANE_DEPARTURE|ADAS_LAST_UPDATE_DATE|ADAS_PERSON_DETECTION|ADAS_RPM|ADAS_SPEED| ANGLE|          BAND_ACC|BAND_ACC_LAST_UPDATE_DATE|BAND_HB|BAND_HB_LAST_UPDATE_DATE|     BUS_COMPANY|      BUS_NO|BUS_ROUTE_ID|BUS_ROUTE_NO|BUS_VH_ID|CO2_DENSITY|    COLLECTION_DA

In [ ]:
test_df.write.json(path="", mode="append", header=True, sep=",", quote='"', lineSep='\n', escapeQuotes=True)

In [ ]:
rddFromFile = spark.sparkContext.textFile("")

In [ ]:
df = spark.read\
    .option("header","true")\
    .format("json")\
    .json(rddFromFile)

In [ ]:
df1 = spark.read.option("header","true").format("json").json(rddFromFile)

In [ ]:
def table_sort(table_name):
    rddFromFile = spark.sparkContext.textFile("")
    df1 = spark.read.option("header","true").format("json").json(rddFromFile)
    df1 = df1.filter(df1.COLLECTION_STATE.contains('SUCCESS'))
    values = table_dict[table_name]
    for v in values:
        if v is 'WORK_DATE':
            df1 = df1.withColumn('WORK_DATE', F.lit(pd.to_datetime(str(datetime.datetime.now())[:19])))
        elif v not in df.columns:
            df1 = df1.withColumn(v,  F.lit(''))
    
    return df1.select(values)

In [ ]:
table_sort('TRACK_SPEED').show()

+-----------------+------+------------+--------------------+------------------+------------------+------+-------+---------+-----------+
|       DEVICE_KEY|MDT_ID|BUS_ROUTE_ID|     COLLECTION_DATE|          LATITUDE|         LONGITUDE| SPEED|MAX_SPD|BUS_VH_ID|DRIVE_COUNT|
+-----------------+------+------------+--------------------+------------------+------------------+------+-------+---------+-----------+
|00-A0-C6-00-0F-4F|   592|   405133202|2021-10-26 19:04:...|33.489969697064616|126.49059919945441| 0.045|       |  7983099|           |
|00-A0-C6-00-48-39|   545|   405578312|2021-10-26 19:04:...| 33.35931718388046|126.33521803968677|52.799|       |  7996035|           |
|00-A0-C6-00-8A-FD|   759|   406262404|2021-10-26 19:04:...|33.260374784239595|126.61758861905771| 0.016|       |  7997541|           |
|00-A0-C6-00-B9-16|   221|   405328206|2021-10-26 19:04:...|33.493188503986914|126.53833199157702|30.562|       |  7983617|           |
|00-A0-C6-01-06-92|   684|   405325107|2021-10-2

In [ ]:
#  df_org[where(col('SPEED') > 70.0)]
df_org[df_org['SPEED'] > 70.0].SPEED

36     70.239
101    96.042
117    82.442
226    70.653
263    81.080
        ...  
627    73.366
652    76.007
698    72.394
858    98.853
893    70.952
Name: SPEED, Length: 4067, dtype: float64